In [1]:
import pandas as pd

#google cloud big query libaries
from google.cloud import bigquery
from google.oauth2 import service_account

import os

In [2]:
dirShiny = "E:/GitHub/Resources/R-Shiny/HTS-Trip-Lengths/input"

group_by_columns = ['groupSampleSegment','groupNumWorkers','groupNumVehicles','groupTripType','groupModeTypeBroad','binSize']
display(group_by_columns)

index_columns = group_by_columns + ['binStart']
display(index_columns)

['groupSampleSegment',
 'groupNumWorkers',
 'groupNumVehicles',
 'groupTripType',
 'groupModeTypeBroad',
 'binSize']

['groupSampleSegment',
 'groupNumWorkers',
 'groupNumVehicles',
 'groupTripType',
 'groupModeTypeBroad',
 'binSize',
 'binStart']

# Setup Data

In [3]:
# Link to BigQuery Client through API

key_path = r"C:\Users\bhereth\confidential-2023-utah-hts-db5335615978.json"
#key_path = r"C:\Users\bhereth\tdm-scenarios-a85044dbbfd3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
print ('Successfully Linked to BigQuery Client!')

Successfully Linked to BigQuery Client!


In [4]:
str_sql = """
SELECT
  bins.binSize,
  bins.binStart,
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles,
  gTt.groupTripType,
  gMtb.groupModeTypeBroad,
  COUNT(*) as numTripRecords,
  SUM(trip.trip_weight) AS sumTripWeight
FROM
  `confidential-2023-utah-hts.bins.binsDist` as bins,
  `confidential-2023-utah-hts.20230313.hh` as hh,
  `confidential-2023-utah-hts.20230313.trip` as trip,
  `confidential-2023-utah-hts.20230313_groupings.groupSampleSegment` as gSs,
  `confidential-2023-utah-hts.20230313_groupings.groupNumWorkers` as gNw,
  `confidential-2023-utah-hts.20230313_groupings.groupNumVehicles` as gNv,
  `confidential-2023-utah-hts.20230313_groupings.groupModeTypeBroad` as gMtb,
  `confidential-2023-utah-hts.20230313_groupings.groupTripType` as gTt
WHERE
  trip.duration_minutes >= bins.binStart AND
  trip.duration_minutes < bins.binStart + bins.binSize AND
  hh.sample_segment = gSs.sample_segment AND
  hh.hh_id = trip.hh_id AND
  hh.num_workers = gNw.num_workers AND
  hh.num_vehicles = gNv.num_vehicles AND
  trip.trip_type = gTt.trip_type AND
  trip.mode_type_broad = gMtb.mode_type_broad
GROUP BY
  bins.binSize,
  bins.binStart,
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles,
  gTt.groupTripType,
  gMtb.groupModeTypeBroad
"""

In [5]:
# test connection
df_trips_by_duration_bins = client.query(str_sql).to_dataframe()

display(df_trips_by_duration_bins)

,binSize,binStart,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupTripType,groupModeTypeBroad,numTripRecords,sumTripWeight
0,0.5,3.0,Al,1,-1,1001,3,169,20666.207588
1,0.5,3.0,Al,0,-1,1001,3,15,1075.231183
2,0.5,24.0,Al,-1,4,1001,-1,5,570.521648
3,0.5,24.0,Al,-1,4,3,3,3,0.000000
4,0.5,24.0,Al,0,-1,-1,-1,5,785.719712
...,...,...,...,...,...,...,...,...,...
2000231,0.5,95.0,TT,-1,1,3,-1,1,0.000000
2000232,0.5,95.0,WF,1,1,1001,1,1,0.000000
2000233,0.5,95.0,Sa,3,-1,1,-1,1,0.000000
2000234,0.5,95.0,Mo,-1,-1,1,3,1,0.000000


In [6]:
_df = df_trips_by_duration_bins.copy()

_df = _df[(_df['groupNumWorkers']==-1) &
     (_df['groupNumVehicles']==-1) &
     (_df['groupTripType']==-1) &
     (_df['binSize']==1.0) &
     (_df['binStart']==0.0)]

display(_df)
display(_df.groupby(['groupModeTypeBroad'], as_index=False).agg(tripcount=('numTripRecords','sum')))

,binSize,binStart,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupTripType,groupModeTypeBroad,numTripRecords,sumTripWeight
18356,1.0,0.0,Al,-1,-1,-1,5,1,0.000000
50982,1.0,0.0,Al,-1,-1,-1,995,53,3583.520455
73821,1.0,0.0,Bo,-1,-1,-1,1,87,4031.035140
145224,1.0,0.0,Ca,-1,-1,-1,-1,1466,21723.288727
145591,1.0,0.0,Ca,-1,-1,-1,5,11,175.583019
...,...,...,...,...,...,...,...,...,...
1247402,1.0,0.0,Un,-1,-1,-1,1,687,37394.326865
1249682,1.0,0.0,Un,-1,-1,-1,-1,819,47671.774124
1395809,1.0,0.0,Ut,-1,-1,-1,4,13,859.701619
1875942,1.0,0.0,We,-1,-1,-1,1,214,10240.603082


,groupModeTypeBroad,tripcount
0,-1,41048
1,1,14382
2,2,497
3,3,17443
4,4,123
5,5,534
6,995,8069
7,1001,15002


# Calculate Distributions

In [7]:
# Group by index and sum 'trip_weight_sum'
df_grouped = df_trips_by_duration_bins.groupby(index_columns).agg({'numTripRecords': 'sum', 'sumTripWeight': 'sum'}).reset_index()

# Calculate the total weight for each trip type
total_weights = df_grouped.groupby(group_by_columns)['sumTripWeight'].transform('sum')

# Calculate percentage distribution
df_grouped['pctTripWeight'] = (df_grouped['sumTripWeight'] / total_weights) * 100

# Add cumulative distribution
df_grouped['cumPctTripWeight'] = df_grouped.groupby(group_by_columns)['pctTripWeight'].cumsum()

df_grouped = df_grouped.fillna(0)

display(df_grouped)

,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupTripType,groupModeTypeBroad,binSize,binStart,numTripRecords,sumTripWeight,pctTripWeight,cumPctTripWeight
0,Al,-1,-1,-1,-1,0.5,0.0,360,22580.656397,3.144299,3.144299
1,Al,-1,-1,-1,-1,0.5,1.0,337,21666.590969,3.017018,6.161317
2,Al,-1,-1,-1,-1,0.5,2.0,544,44266.100742,6.163942,12.325259
3,Al,-1,-1,-1,-1,0.5,3.0,691,47808.823860,6.657257,18.982517
4,Al,-1,-1,-1,-1,0.5,4.0,605,53213.739358,7.409878,26.392395
...,...,...,...,...,...,...,...,...,...,...,...
2000231,We,8,7,1002,3,1.0,34.0,1,90.183729,13.394053,100.000000
2000232,We,8,7,1002,3,5.0,5.0,2,148.397326,22.039914,22.039914
2000233,We,8,7,1002,3,5.0,10.0,1,90.183729,13.394053,35.433967
2000234,We,8,7,1002,3,5.0,25.0,3,254.363207,37.777927,73.211894


In [9]:
df_grouped['sumTripWeight'] = df_grouped['sumTripWeight'].round(2)
df_grouped['pctTripWeight'] = df_grouped['pctTripWeight'].round(2)
df_grouped['cumPctTripWeight'] = df_grouped['cumPctTripWeight'].round(2)

df_grouped.to_csv('../input/hts-trip-durations.csv', index=False)

In [10]:
df_value_labels= client.query("SELECT * FROM `confidential-2023-utah-hts.20230313.value_labels`").to_dataframe()

display(df_value_labels)

,table,variable,value,label
0,person,age,8,55-64
1,person,age,1,Under 5
2,person,age,2,5-15
3,person,age,11,85 or older
4,person,age,3,16-17
...,...,...,...,...
2037,person,second_home_county,995,Missing response
2038,trip,trace_quality_flag,995,Missing response
2039,person,commute_subsidy_998,995,Missing response
2040,trip,trip_survey_complete,995,Missing response


In [11]:
def process_labels(variable, new_column_name):
    """
    Processes and formats a specific variable's labels from a DataFrame.

    Parameters:
    variable (str): The variable to filter and process.
    new_column_name (str): The new column name for the processed variable.

    Returns:
    DataFrame: A processed DataFrame with formatted labels for the specified variable.
    """
    # Filter the DataFrame for the specified variable
    df_labels = df_value_labels[df_value_labels['variable'] == variable].copy()
    
    # Assuming df_labels is your DataFrame and 'value' is initially not in integer format
    df_labels['value'] = df_labels['value'].astype(int)  # Convert 'value' to integer

    # Now sort the DataFrame by the 'value' column
    df_labels.sort_values(by='value', inplace=True)

    # Rename 'value' column to new_column_name and convert it to int64
    #df_labels.rename(columns={'value': variable}, inplace=True)

    #df_labels['newLabel'] = df_labels['value'].astype(str) + ': ' + df_labels['label']
    
    df_labels['newLabel'] = df_labels['label']

    # Drop unnecessary columns
    df_labels.drop(columns=['table', 'variable', 'label'], inplace=True)

    df_labels.rename(columns={'newLabel': 'label'}, inplace=True)

    
    df_labels = pd.concat([pd.DataFrame([[-1,'All']], columns=['value', 'label']),df_labels])

    display(df_labels)

    return df_labels

# Example usage:
# Assuming df_value_labels is a DataFrame you have that meets the requirements
df_trip_type_labels = process_labels('trip_type', 'Trip Type')
df_mode_type_broad_labels = process_labels('mode_type_broad', 'Broad Mode Type')
df_num_workers_labels = process_labels('num_workers', 'Number Worker')
df_num_vehicles_labels = process_labels('num_vehicles', 'Number Vehicle')


,value,label
0,-1,All
772,1,Home-based work
769,2,Home-based school
773,3,Home-based shopping
768,4,Home-based personal business
774,5,Home-based other
770,6,Non-home-based work
771,7,Non-home-based non-work
2003,995,Missing response


,value,label
0,-1,All
1244,1,Walk
1245,2,Bike
1247,3,Car
1246,4,Transit
1546,5,Other
2030,995,Missing response


,value,label
0,-1,All
908,0,0 (No workers)
911,1,1 worker
909,2,2 workers
917,3,3 workers
907,4,4 workers
918,5,5 workers
915,6,6 workers
912,7,7 workers
910,8,8 workers


,value,label
0,-1,All
1050,0,0 (no vehicles in household)
1049,1,1 vehicle
1052,2,2 vehicles
1047,3,3 vehicles
1051,4,4 vehicles
1053,5,5 vehicles
1048,6,6 vehicles
1045,7,7 vehicles
1046,8,8 or more vehicles
